In [1]:
import sympy as sym
import numpy as np

In [2]:
p_ta,p_tb,p_tc = sym.symbols('p_ta,p_tb,p_tc')
p_va,p_vb,p_vc = sym.symbols('p_va,p_vb,p_vc')
p_ra,p_rb,p_rc = sym.symbols('p_ra,p_rb,p_rc')
p_la,p_lb,p_lc = sym.symbols('p_la,p_lb,p_lc')
p_v = sym.Symbol('p_v')

In [3]:
eq_a = p_ta + p_va - p_la - p_ra
eq_b = p_tb + p_vb - p_lb #- p_rb
eq_c = p_tc + p_vc - p_lc #- p_rc
eq_v = -p_v + p_va + p_vb + p_vc 
eq_t_ab = p_ta - p_tb 
eq_t_ac = p_ta - p_tc 
eq_l_bc = p_lb - p_lc 

In [5]:
sol = sym.solve([eq_a,eq_b,eq_c,eq_v,eq_t_ab,eq_t_ac,eq_l_bc],[p_va,p_vb,p_vc,p_la,p_lb,p_lc])

sol

{p_va: -2*p_lc + p_tb + p_tc + p_v,
 p_vb: p_lc - p_tb,
 p_vc: p_lc - p_tc,
 p_la: -2*p_lc - p_ra + p_ta + p_tb + p_tc + p_v,
 p_lb: p_lc}

In [27]:
p_l = p_l_a + p_l_b + p_l_c
p_r = p_r_a + p_r_b + p_r_c
p_t = p_t_a + p_t_b + p_t_c
p_v = p_v_a + p_v_b + p_v_c

p_l_a + p_l_b + p_l_c + p_r_a + p_r_b + p_r_c + 3*p_ta + p_v 

{p_va: -2*p_lc + p_tb + p_tc + p_v,
 p_vb: p_lc - p_tb,
 p_vc: p_lc - p_tc,
 p_la: -2*p_lc - p_ra + p_ta + p_tb + p_tc + p_v,
 p_lb: p_lc}

In [35]:
#p_t + p_v - p_l - p_r = 0
p_vc = p_vb
p_va = p_v - p_vb - p_vc
p_ta  = -p_va + p_la + p_ra

eq = 3*p_ta + p_va + p_vb + p_vc - p_la - p_lb - p_lc - p_ra 
sol = sym.solve(eq,p_vb)
sol

[-p_la/3 + p_lb/6 + p_lc/6 - p_ra/3 + p_v/3]

## Equilibrador de potencia compleja para transformadores

En el caso del sistema de dos lineas AC y dos lineas DC (2w2w), el transformador se puede ver sometido a un desequilibrio extremo. Esto se debe a que la carga AC puede ser puramente monofásica. Por este motivo se propone que los convertidores de cabecera equilibren las fases del transoformador inyectando corrientes desequilibradas.


<img src="../docs/trafo_equalizer.png" alt="Drawing" style="width: 400px;"/>

La salida de potencia del transformador será igual a las cargas trifásicas $p_{la}$, se considera que se tienen una
    T_p = 0.5

$$
s_{lb} = s_{tb} + s_{vb}
$$
   
$$
s_{ra} = s_{ta} + s_{va} - s_{lb}
$$

\begin{eqnarray}
    s_{vb}^\star &=& (s_v-s_{ra})/3 \\
    s_{vc}^\star &=& s_{vb} \\
    s_{va}^\star &=& s_v - 2 s_{vb} 
\end{eqnarray}   

\begin{eqnarray}
    s_{va} &=& s_{va} + \frac{\Delta t}{T_p} \left(s_{va}^\star - s_{va}\right)\\
    s_{vb} &=& s_{vb} + \frac{\Delta t}{T_p} \left(s_{vb}^\star - s_{vb}\right)\\
    s_{vc} &=& s_{vc} + \frac{\Delta t}{T_p} \left(s_{vc}^\star - s_{vc}\right) 
\end{eqnarray}    
    
    
    
    p_lb = p_tb + p_vb
    p_ra = p_ta + p_va - p_lb
    
    p_vb_ref = (p_v-p_ra)/3 
    p_vc_ref = p_vb
    p_va_ref = p_v - 2*p_vb
    
    p_va = p_va + Dt/T_p*(p_va_ref - p_va)
    p_vb = p_vb + Dt/T_p*(p_vb_ref - p_vb)
    p_vc = p_vc + Dt/T_p*(p_vc_ref - p_vc)



In [4]:
def balancer(s_v,s_ta,s_tb,s_tc,s_va,s_vb,s_vc,Dt):
    '''
    Se equilibran las potencias de cada fase de los transformadores con el VSC de cabecera.
    p_v: potencia total a entregar por el VSC
    
    p_ta,p_tb,p_tc: potencias por fase del transformador
    p_va,p_vb,p_vc: potencias por fase, se debe cumplir que p_v = p_va + p_vb + p_vc

    '''
    T_p = 5*Dt
    
    s_lb = s_tb + s_vb
    s_ra = s_ta + s_va - s_lb
    
    s_vb_ref = (s_v-s_ra)/3 
    s_vc_ref = s_vb
    s_va_ref = s_v - 2*s_vb
    
    s_va = s_va + Dt/T_p*(s_va_ref - s_va)
    s_vb = s_vb + Dt/T_p*(s_vb_ref - s_vb)
    s_vc = s_vc + Dt/T_p*(s_vc_ref - s_vc)

    return s_va,s_vb,s_vc
    

### Test

In [5]:
p_va,p_vb,p_vc = 0,0,0
Dt_mid = 0.1
p_v = -145 

times = np.arange(0,5,Dt_mid)
for t in times:
    p_ra = 0
    p_la,p_lb,p_lc =200/3,200/3,200/3

    p_ta = -p_va + p_la + p_ra
    p_tb = -p_vb + p_lb #- p_rb
    p_tc = -p_vc + p_lc #- p_rc

    #print(f'{p_ta},{p_tb},{p_tc}')

     
    p_va,p_vb,p_vc = balancer(p_v,p_ta,p_tb,p_tc,p_va,p_vb,p_vc,Dt_mid)
    
        
    print(f'{p_va},{p_vb},{p_vc}; {p_va+p_vb+p_vc}')
    #print(f'{p_ta},{p_tb},{p_tc}')


-29.0,-9.666666666666668,0.0; -38.66666666666667
-48.33333333333333,-17.400000000000002,-1.9333333333333336; -67.66666666666667
-60.70666666666666,-23.58666666666667,-5.026666666666667; -89.32000000000001
-68.13066666666666,-28.536,-8.738666666666667; -105.40533333333332
-72.09013333333333,-32.495466666666665,-12.698133333333335; -117.28373333333333
-73.67392,-35.66304,-16.657600000000002; -125.99456
-73.67392,-38.19709866666667,-20.458688000000002; -132.32970666666665
-72.66029653333332,-40.2243456,-24.006370133333334; -136.89101226666668
-71.03849898666665,-41.84614314666667,-27.249965226666667; -140.13460736
-69.09234193066665,-43.143581184000006,-30.169200810666666; -142.40512392533333
-67.01644107093331,-44.181531613866674,-32.76407688533333; -143.96204957013333
-64.94054021119997,-45.01189195776001,-35.04756783104; -144.99999999999997
-62.94767538585597,-45.67618023287467,-37.040432656384; -145.66428827511464
-61.08766821553491,-46.207610852966404,-38.767582171682136; -146.062861

In [209]:
p_vc = p_vb
p_l = p_la + p_lb + p_lc
#p_v = p_va + p_vb + p_vc

p_t = p_l + p_ra - p_va + p_vb + p_vc
#p_ta = p_t/3
#p_tb = p_t/3
#p_tc = p_t/3

eq_a = -p_ta + p_t/3
eq_b = -p_v + p_va + p_vb + p_vc

In [66]:
sym.solve([eq_a,eq_b],p_va,p_vb)

{p_va: p_la/2 + p_lb/2 + p_lc/2 + p_ra/2 - 3*p_ta/2 + p_v/2,
 p_vb: -p_la/4 - p_lb/4 - p_lc/4 - p_ra/4 + 3*p_ta/4 + p_v/4}

In [41]:
def equalizer(s_t_res, s_t_ind, s_t_com,s_v_res, s_v_ind, s_v_com,S_res,S_ind,S_com):
    '''
    s = p + jq
    s_t_res, s_t_ind, s_t_com: potencias totales medidas a la salida de cada trafo
    s_l_res, s_l_ind, s_l_com: carga conectada a cada trafo
    s_v_res, s_v_ind, s_v_com: potencia de VSC de cabecera
    S_res,S_ind,S_com: potencias nominales/base/maximas de cada trafo
    
    
    s_t = s_l - s_v
    
    '''
       
    s_total = (s_res + s_ind + s_com)
    
    s_res_ref = s_total*S_res/(S_res + S_ind + S_com) # potencias deseadas 
    s_ind_ref = s_total*S_ind/(S_res + S_ind + S_com) # potencias deseadas
    s_com_ref = s_total*S_com/(S_res + S_ind + S_com) # potencias deseadas
    
    #s_l = s_t + s_v, s_v del paso anterior
    #s_v = s_l - s_ref, con s_v la nueva potencia
    
    s_l_res = s_t_res + s_v_res 
    s_l_ind = s_t_ind + s_v_ind 
    s_l_com = s_t_com + s_v_com 

    s_v_res = s_l_res - s_res_ref
    s_v_ind = s_l_ind - s_ind_ref
    s_v_com = s_l_com - s_com_ref  
    
    return s_v_res,s_v_ind,s_v_com



### Test

In [45]:
p_res = 100e3
p_ind = 100e3
p_com = 100e3
q_res = 0e3
q_ind = 0e3
q_com = 0e3
S_res = 400e3
S_ind = 160e3
S_com = 250e3

s_l_res = 100 + 1j*10
s_l_ind = 100 + 1j*10
s_l_com = 100 + 1j*10

s_v_res = 0 + 1j*0
s_v_ind = 0 + 1j*0
s_v_com = 0 + 1j*0

for it in range(10):
    
    s_t_res = s_l_res - s_v_res 
    s_t_ind = s_l_ind - s_v_ind 
    s_t_com = s_l_com - s_v_com     
    

    print(f'{s_t_res.real/1e3:4.0f}, {s_t_ind.real/1e3:4.0f}, {s_t_com.real/1e3:4.0f}')

    s_v_res,s_v_ind,s_v_com = equalizer(s_t_res, s_t_ind, s_t_com,s_v_res, s_v_ind, s_v_com,S_res,S_ind,S_com)

    
    
    
    
s_total_ref = s_res_ref + s_ind_ref + s_com_ref
s_total_ref

   0,    0,    0
 148,   59,   93
 148,   59,   93
 148,   59,   93
 148,   59,   93
 148,   59,   93
 148,   59,   93
 148,   59,   93
 148,   59,   93
 148,   59,   93


(300000+0j)

In [58]:
def equalizer_dc(p_v_res_dc, p_v_ind_dc, p_v_com_dc,p_v_res_ac, p_v_com_ac,S_res,S_ind,S_com,Dt):
    '''
    Siendo VSC IND grid-former se definen las referencias de potencia para VSCs RES y COM 
    de manera que los tres den una potencia en por unidad igual al bus DC.
    
    p_v_res_dc, p_v_ind_dc, p_v_com_dc: potencias totales medidas a la salida DC de cada convertidor
    p_v_res_ac, p_v_com_ac: potencia AC de referencia para los grid feeders de cabecera   
    
    
    '''
    
    T_p = 0.1
       
    p_dc = p_v_res_dc + p_v_ind_dc + p_v_com_dc
    
    p_v_res_ac += Dt/T_p*(p_dc*S_res/(S_res + S_ind + S_com) - p_v_res_ac) # potencias deseadas en AC
    p_v_com_ac += Dt/T_p*(p_dc*S_ind/(S_res + S_ind + S_com) - p_v_com_ac) # potencias deseadas en AC
    
    return p_v_res_ac,p_v_com_ac

### Test

In [69]:


S_res =  50
S_ind = 100
S_com = 100

p_l_res_dc = 50
p_l_ind_dc = 50
p_l_com_dc = 50

p_v_res_ac = 0
p_v_com_ac = 0

p_l = p_l_res_dc + p_l_ind_dc + p_l_com_dc  
p_ind_dc = p_l - p_v_res_ac - p_v_res_ac
  
p_v_res_dc =    0
p_v_com_dc =    0
    
Dt = 0.01

times = np.arange(0,5,Dt)
for t in times:
    
    p_l = p_l_res_dc + p_l_ind_dc + p_l_com_dc  # total DC
    p_v_ind_dc = p_l - p_v_res_dc - p_v_com_dc    

    print(f't={t:0.1f}, p_res = {p_v_res_dc:3.0f}, p_ind = {p_v_ind_dc:3.0f}, p_com = {p_v_com_dc:3.0f}')

    p_v_res_ac,p_v_com_ac = equalizer_dc(p_v_res_dc, p_v_ind_dc, p_v_com_dc,p_v_res_ac, p_v_com_ac,S_res,S_ind,S_com,Dt)
    
    p_v_res_dc = p_v_res_ac
    p_v_com_dc = p_v_com_ac
    
    
    
    
#s_total_ref = s_res_ref + s_ind_ref + s_com_ref
#s_total_ref

t=0.0, p_res =   0, p_ind = 150, p_com =   0
t=0.0, p_res =   3, p_ind = 141, p_com =   6
t=0.0, p_res =   6, p_ind = 133, p_com =  11
t=0.0, p_res =   8, p_ind = 126, p_com =  16
t=0.0, p_res =  10, p_ind = 119, p_com =  21
t=0.1, p_res =  12, p_ind = 113, p_com =  25
t=0.1, p_res =  14, p_ind = 108, p_com =  28
t=0.1, p_res =  16, p_ind = 103, p_com =  31
t=0.1, p_res =  17, p_ind =  99, p_com =  34
t=0.1, p_res =  18, p_ind =  95, p_com =  37
t=0.1, p_res =  20, p_ind =  91, p_com =  39
t=0.1, p_res =  21, p_ind =  88, p_com =  41
t=0.1, p_res =  22, p_ind =  85, p_com =  43
t=0.1, p_res =  22, p_ind =  83, p_com =  45
t=0.1, p_res =  23, p_ind =  81, p_com =  46
t=0.1, p_res =  24, p_ind =  79, p_com =  48
t=0.2, p_res =  24, p_ind =  77, p_com =  49
t=0.2, p_res =  25, p_ind =  75, p_com =  50
t=0.2, p_res =  25, p_ind =  74, p_com =  51
t=0.2, p_res =  26, p_ind =  72, p_com =  52
t=0.2, p_res =  26, p_ind =  71, p_com =  53
t=0.2, p_res =  27, p_ind =  70, p_com =  53
t=0.2, p_r

In [55]:
equalizer_dc(p_v_res_dc, p_v_ind_dc, p_v_com_dc,p_v_res_ac, p_v_com_ac,S_res,S_ind,S_com)

TypeError: unsupported operand type(s) for /: 'tuple' and 'int'